In [1]:
import os 
import io
import math
import torch 
from torchvision import transforms
from torchvision.utils import save_image
import numpy as np
import time
from PIL import Image
import onnxruntime
import onnx

In [2]:
def preprocess_gray(image_path):
    image=Image.open(image_path).convert("L")
    #resize the image
    image=image.resize((1640,1232))
    #convert to numpy and normalize
    image=np.array(image)/255.0
    
    #create an empty numpy array
    img_blank=np.zeros((3,1232,1640))

    #assign the gray image to the blank array
    img_blank[0]=image
    
    
    img_final=np.expand_dims(img_blank,axis=0).astype(np.float32)

   
    return img_final


In [3]:
def inference_setup(model_path):
    #load the ONNX model
    session_options=onnxruntime.SessionOptions()
    session_options.graph_optimization_level=onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL

    #Check if CUDA is available
    providers = [('CUDAExecutionProvider',{"use_tf32":0})] if 'CUDAExecutionProvider' in onnxruntime.get_available_providers() else ['CPUExecutionProvider']
    print("Using:",providers)
    onnx_session = onnxruntime.InferenceSession(model_path,sess_options=session_options, providers=providers)

    input_names=["input"]
    output_names=["output"]

    return onnx_session,input_names,output_names

def run_infer(onnx_session,input_image,output_names,input_names):
    onnx_output = onnx_session.run(output_names, {input_names[0]: input_image})[0]
    return onnx_output
    
    

In [4]:
def run_inference( model_path):
    """
    Run inference on an input image using the exported ONNX model.
    """
    
    # Load the ONNX model

    session_options = onnxruntime.SessionOptions()
    session_options.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
    #session_options.log_severity_level = onnxruntime.logging.LoggingLevel.WARNING

    #Check if CUDA is available
    providers = [('CUDAExecutionProvider',{"use_tf32":0})] if 'CUDAExecutionProvider' in onnxruntime.get_available_providers() else ['CPUExecutionProvider']
    print("Using:",providers)
    onnx_session = onnxruntime.InferenceSession(model_path,sess_options=session_options, providers=providers)
    #onnx_session = onnxruntime.InferenceSession(model_path)

    
    # Run inference for color image
    input_names = ["input"]
    output_names = ["output"]
    start=time.time()
    onnx_output = onnx_session.run(output_names, {input_names[0]: input_image})[0]
    end=time.time()

    print("Time in seconds for the inference is:",round(end-start,2),"seconds")

    #run inference for the grayscale image
    input_names=["input"]
    output_names=["output"]
    start=time.time()
    onnx_output_gray=onnx_session.run(output_names,{input_names[0]:input_image_gray})[0]
    end=time.time()

    print("Time in seconds for the inference of gray is:",round(end-start,2),"seconds")
    
    return onnx_output, onnx_output_gray
    
 


In [5]:
def save_compressed_image(output, save_path):
    """
    Save the compressed output image from the model inference.
    """
    # Post-process the output (if needed)
    output = output.squeeze(0)  # Remove batch dimension
    
    # Convert to HWC format (Height x Width x Channels)
    output = np.transpose(output, (1, 2, 0))
    
    # Clip values to valid range (0, 1) and convert to 8-bit (0-255)
    output = np.clip(output, 0, 1) * 255.0
    output = output.astype(np.uint8)
    
    # Convert to image format and save
    output_image = Image.fromarray(output)
    output_image.save(save_path) 


def export_to_buffer(output):

    #post process the output
    output=output.squeeze(0)

    #convert to the HWC format
    output=np.transpose(output,(1,2,0))

    #clip values to the valid range(0,1)
    output=np.clip(output,0,1)*255.0
    output=output.astype(np.uint8)

    #convert to PIL image and write the data to the in_memory buffer
    output_image=Image.fromarray(output)
    buffer_stream=io.BytesIO()
    output_image.save(buffer_stream,format="JPEG")

    return buffer_stream.getvalue()


In [6]:
   # Save the output image
def exporting_output(onnx_output,onnx_output_gray,save_path):
    start=time.time()
    save_compressed_image(onnx_output, save_path)
    end=time.time()
    print("Exporting to disc timing:",round(end-start,2),"seconds")
    
    start=time.time()
    buffer=export_to_buffer(onnx_output)
    end=time.time()
    print("Exporting to the buffere timing:",round(end-start,2),"seconds")
    print(f"Compressed image saved at: {save_path}")

In [7]:
image_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\FLOPS\image.png"
model_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\FLOPS\bmshj2018_factorized_models\bmshjRELU_halfUHD_ssim_8.onnx"
save_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\FLOPS\PyT_bmshjRELU_halfUHD_ssim_8.png"
dataset_path=r"C:\Swapnil\Narrowband_DRONE\atsugi_dataset_small_50\uncomp_images"
export_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\FLOPS\ONNX_pytorch_dataset\onnx_compressed_images"


In [8]:
def analyze_time(preprocess,infer,buff):
    preprocess=np.array(preprocess)
    infer=np.array(infer)
    buff=np.array(buff)
    print("Average time for infer per image:",np.mean(infer))
    print("Average time for preprocess per image:",np.mean(preprocess))
    print("Average time for buff per image:",np.mean(buff))
    print("average FPS achievable is:",int(1/np.mean(infer)))

In [9]:
def image_handling(dataset_path):
    lst=os.listdir(dataset_path)
    images=[]
    for image in lst:
        if image.lower().endswith("jpg") or image.lower().endswith("jpeg") or image.lower().endswith("png") or image.lower().endswith("webp"):
            images.append(image)
    print("dataset has:",len(images),"images")

    return images
    
    

In [ ]:
def main(model_path,dataset_path):

    #get the images dataset
    images=image_handling(dataset_path)

    #Doing for the Gray only

    #setting up for the inference
    onnx_session,input_names,output_names=inference_setup(model_path)

    preprocess_time=[]
    infer_time=[]
    buff_export_time=[]
    
    for image in images:
        image_path=os.path.join(dataset_path,image)
        #preprocess the image as grayscale
        start=time.time()
        input_image=preprocess_gray(image_path)
        #print("input image type:",type(input_image),"input_image:",input_image.shape)
        #print("image is like:",input_image[0])
        end=time.time()
        preprocess_time.append(round(end-start,2))
        print("preprocess time:",round(end-start,2))
        
        #run the inference
        start=time.time()
        onnx_output=run_infer(onnx_session,input_image,output_names,input_names)
        end=time.time()
        infer_time.append(round(end-start,2))
        print("infer time:",round(end-start,2))
        
        #export to the buffer
        start=time.time()
        export_to_buffer(onnx_output)
        end=time.time()
        buff_export_time.append(round(end-start,2))
        print("Done for image:",image)
        print("export time:",round(end-start,2))
        buffer.truncate(0)

    analyze_time(preprocess_time,infer_time,buff_export_time)
    
    
        
        
        

main(model_path,dataset_path)